In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

class MLP(nn.Module):
    def __init__(self,input,output):
        super().__init__()
        self.layer1 = nn.Linear(input, 16)
        self.layer2 = nn.Linear(16,20)
        self.layer3 = nn.Linear(20,output)

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [2]:
import torch

class DataSet(torch.utils.data.Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx], self.labels[idx]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.genfromtxt('iris.data',delimiter=',')
data = np.delete(data,-1,axis=1)
classes = np.genfromtxt('iris.data',delimiter=',', dtype=str,usecols=[-1])
classes = np.unique(classes,return_inverse=1)[1]

data_treino, data_teste, classe_treino, classe_teste = train_test_split(data,classes,test_size=0.2,stratify=classes,random_state=123)

data_treino, data_validacao, classe_treino, classe_validacao = train_test_split(data_treino,classe_treino,test_size=0.1, stratify=classe_treino, random_state=1)

dataset = DataSet(data_treino,classe_treino)
data_train = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

dataset = DataSet(data_validacao,classe_validacao)
data_val = DataLoader(dataset, batch_size=len(data_validacao), shuffle=True, num_workers=0)

dataset = DataSet(data_teste,classe_teste)
data_test = DataLoader(dataset, batch_size=len(data_teste), shuffle=True, num_workers=0)

In [4]:
mlp = MLP(4,3)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [5]:
def train_one_epoch(NeuralNet,Loss,Optimizer,data_treino):
    current_loss = 0.0
    total_samples = 0

    for data in data_treino:
        x,y = data
        x = x.to(torch.float)
        Optimizer.zero_grad()
        output = NeuralNet(x)
        loss = Loss(output,y)
        loss.backward()
        Optimizer.step()
        current_loss += loss.item()
        total_samples += len(x)

    return current_loss/total_samples

In [6]:
epochs = 5000

mlp.train(True)
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}')

    loss_per_epoch = train_one_epoch(mlp,loss_function,optimizer,data_train)
    
    print(f'Loss: {loss_per_epoch}')

print('Training process has finished.')

Starting epoch 1
Loss: 0.04107671755331534
Starting epoch 2
Loss: 0.04086446872463933
Starting epoch 3
Loss: 0.04077506617263511
Starting epoch 4
Loss: 0.040859211374212195
Starting epoch 5
Loss: 0.04109639591640896
Starting epoch 6
Loss: 0.040714760621388756
Starting epoch 7
Loss: 0.040804380619967426
Starting epoch 8
Loss: 0.040694615355244386
Starting epoch 9
Loss: 0.04053407024454187
Starting epoch 10
Loss: 0.04096156027581957
Starting epoch 11
Loss: 0.0406820034539258
Starting epoch 12
Loss: 0.04073537610195301
Starting epoch 13
Loss: 0.04067884882291158
Starting epoch 14
Loss: 0.040748171232364794
Starting epoch 15
Loss: 0.04058447480201721
Starting epoch 16
Loss: 0.04043218383082637
Starting epoch 17
Loss: 0.040691128483525026
Starting epoch 18
Loss: 0.040590119582635385
Starting epoch 19
Loss: 0.040660491696110475
Starting epoch 20
Loss: 0.04041753985263683
Starting epoch 21
Loss: 0.040322436226738825
Starting epoch 22
Loss: 0.04060958601810314
Starting epoch 23
Loss: 0.0404463

In [7]:
from sklearn.metrics import classification_report

mlp.train(False)

for data in data_test:
    x_test,y_test = data
    x_test = x_test.to(torch.float32)
    y_pred = mlp(x_test)
    y_pred = torch.Tensor.argmax(y_pred, dim=1)
    y_pred = torch.Tensor.numpy(y_pred)
    y_test = torch.Tensor.numpy(y_test)

print(classification_report(y_test,y_pred,zero_division=1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

